In [1]:
import tensorflow as tf
import numpy as np

2025-10-03 14:13:33.571698: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-03 14:13:34.056503: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def init_links(Lx: int, Ly: int, random: bool = True):
    """
    Initialize horizontal (hx) and vertical (hy) link spins for Z2 gauge theory.

    Parameters
    ----------
    Lx, Ly : int
        Lattice size (sites).
    random : bool, default True
        If True, initialize links randomly with ±1.
        If False, initialize all to +1.

    Returns
    -------
    hx : np.ndarray of shape (Lx, Ly)
        Horizontal link spins.
    hy : np.ndarray of shape (Lx, Ly)
        Vertical link spins.
    """
    if random:
        hx = np.random.choice([-1, 1], size=(Lx, Ly))
        hy = np.random.choice([-1, 1], size=(Lx, Ly))
    else:
        hx = np.ones((Lx, Ly), dtype=int)
        hy = np.ones((Lx, Ly), dtype=int)

    return hx, hy

def plaquettes_from_links_tf(hx, hy, periodic="xy"):
    """
    Compute Z2 plaquettes from link fields with customizable boundary conditions.

    Parameters
    ----------
    hx : tf.Tensor, shape (Lx, Ly)
        Horizontal link spins (±1).
    hy : tf.Tensor, shape (Lx, Ly)
        Vertical link spins (±1).
    periodic : str or None
        Boundary condition type:
          None  -> open boundaries
          'x'   -> periodic along x-axis (rows)
          'y'   -> periodic along y-axis (cols)
          'xy'  -> periodic along both (torus)

    Returns
    -------
    P : tf.Tensor, shape (Lx, Ly)
        Plaquette values (±1 or 0 if incomplete at boundary).
    Sx, Sy : tf.Tensor
        Intermediate pairwise products along y and x respectively.
    """
    Lx = tf.shape(hx)[0]
    Ly = tf.shape(hx)[1]

    if periodic in ("y", "xy"):
        Sx = hx * tf.roll(hx, shift=-1, axis=1)
    else:
        Sx = tf.concat(
            [hx[:, :-1] * hx[:, 1:], tf.zeros([Lx, 1], dtype=hx.dtype)],
            axis=1
        )

    if periodic in ("x", "xy"):
        Sy = hy * tf.roll(hy, shift=-1, axis=0)
    else:
        Sy = tf.concat(
            [hy[:-1, :] * hy[1:, :], tf.zeros([1, Ly], dtype=hy.dtype)],
            axis=0
        )

    P = Sx * Sy
    return P, Sx, Sy



# Example: 4x4 lattice
hx, hy = init_links(4, 4, random=True)
print("Horizontal links hx:\n", hx)
print("Vertical links hy:\n", hy)


Horizontal links hx:
 [[-1  1 -1  1]
 [-1  1  1  1]
 [-1  1  1 -1]
 [-1  1 -1  1]]
Vertical links hy:
 [[ 1  1  1 -1]
 [-1  1  1 -1]
 [-1 -1 -1  1]
 [ 1  1 -1 -1]]


In [6]:
hx

array([[-1,  1, -1,  1],
       [-1,  1,  1,  1],
       [-1,  1,  1, -1],
       [-1,  1, -1,  1]])

In [7]:
hx

array([[-1,  1, -1,  1],
       [-1,  1,  1,  1],
       [-1,  1,  1, -1],
       [-1,  1, -1,  1]])

In [9]:
P, Sx, Sy = plaquettes_from_links_tf(hx, hy)

In [10]:
P

<tf.Tensor: shape=(4, 4), dtype=int64, numpy=
array([[ 1, -1, -1, -1],
       [-1, -1, -1,  1],
       [ 1, -1, -1, -1],
       [-1, -1,  1, -1]])>